In [25]:
import cifar10
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf

In [26]:
cifar10.data_path = "data/CIFAR-10/"

In [27]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [28]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [29]:
images_train_data = images_train.reshape(50000, 3072)

In [30]:
n_input = 3072
input_width = 32
input_height = 32
input_channels = 3

n_conv1 = 32
n_conv2 = 64
conv1_k = 5
conv2_k = 5

n_hidden = 1024
n_out = 10

pooling_window_size = 2

In [31]:
conv2_out_width = input_width//(pooling_window_size*pooling_window_size)
conv2_out_height = input_height//(pooling_window_size*pooling_window_size) 
dense_layer_input = conv2_out_width*conv2_out_height * n_conv2

In [32]:
weights = {
    'c1': tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    'c2': tf.Variable(tf.random_normal([conv2_k,conv2_k, n_conv1,n_conv2])),
    'd1': tf.Variable(tf.random_normal([dense_layer_input, n_hidden])),
    'out': tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    'c1': tf.Variable(tf.random_normal([n_conv1])),
    'c2': tf.Variable(tf.random_normal([n_conv2])),
    'd1': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_out]))
}

In [33]:
def conv(x, weights, bias, stride = 1):
    conv_out = tf.nn.conv2d(x, weights, padding="SAME", strides=[1,stride,stride,1])
    out = tf.nn.bias_add(conv_out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding="SAME", ksize=[1, k, k, 1], strides = [1, k, k, 1])

In [34]:
def cnn(x, weights, biases):
    x = tf.reshape(x, shape=[-1,input_width, input_height, input_channels ])
    conv1 = conv(x, weights["c1"], biases['c1'])
    conv1 = maxpooling(conv1, k = pooling_window_size)
    
    conv2 = conv(conv1, weights["c2"], biases['c2'])
    conv2 = maxpooling(conv2, k = pooling_window_size)
    
    hidden_input = tf.reshape(conv2, shape=[-1, dense_layer_input])
    hidden_output = tf.add(tf.matmul(hidden_input, weights['d1']), biases['d1'])
    hidden_output = tf.nn.relu(hidden_output)
    
    in_output_layer = tf.add(tf.matmul(hidden_output, weights['out']), biases['out'])
    output = in_output_layer
    return output

In [35]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder(tf.int32, [None, n_out])

In [36]:
pred = cnn(x, weights, biases)

In [37]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))

In [38]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize_step = optimizer.minimize(cost)

In [39]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [40]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


In [41]:
batch_size = 100
for i in range(25):
    num_batches = int(images_train.shape[0]/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = next_batch(batch_size, images_train_data, labels_train)
        c, _ = sess.run([cost, optimize_step], feed_dict={x:batch_x, y:batch_y})
        total_cost += c
    print(total_cost)

3273918.308330536
1463.02751994133
1255.0931062698364


KeyboardInterrupt: 

In [ ]:
predictions = tf.argmax(pred, axis = 1)
actual_labels = tf.argmax(y, axis = 1)
correct_predictions = tf.equal(predictions,actual_labels)

correct_preds = sess.run(correct_predictions, feed_dict={x:images_train_data, y:labels_train} )
correct_preds.sum()